# Week 3 Activity: Synthesis Functions

Complete this activity as part of your participation grade. Pending length of the lecture, you will have time in class to work. Everything you need to complete this activity can be found in this week's (or a previous week's) lecture code. For this activity, you may want to consult your Audio Tech I notes (or the Computer Music Tutorial)

## 1. Create an envelope function

Write a function called adsr() that will modify a passed sound according to an ADSR envelope that is user-defined. 
Your function should include the following arguments at a minimum:

1) a: Time to reach maximum amplitude (can be zero to length of input). (float)

2) s: The level (amplitude) of the sustain portion. (float)

3) d: Time to descend from max to sustain level (N.B. time can be zero, and sustain can be False.) (float)

4) r: Time to descend to zero (N.B. should be a minimum of 20ms). (float)

5) x: array of samples of length n that will be modulated by the envelope. (np.array)

Your function should return the input sound modified by the envelope

**Notes**

Optimizing your function:  The following are things to consider, but not necessary as a "first step":  

* Your function ideally should allow for all scenarios including passing sound arrays of various lengths, and, it should allow for a sharp attack followed by an immediate decay with zero sustain (like a plucked or percussive sound) regardless of the total length of the sound passed. I.e., If there is no sustain value, then the Decay (#3) and Sustain (#2) are moot, so your function should just ascend to peak value over some time, x, and then descend to zero over time x.

* Be sure to include default values to all your arguments (except #5)

* You will need to define additional defaults within the function such as the sample rate. (You may want to make this an argument variable with a default value).


In [ ]:
import numpy as np

def adsr(x: np.ndarray,
         a: float = 0.01,     
         s: float | bool = 0.7, 
         d: float = 0.10,      
         r: float = 0.20,      
         sr: int = 44100       
         ) -> np.ndarray:

    x = np.asarray(x, dtype=float)
    n = x.size
    if n == 0:
        return x


    s_level = float(s)
    s_level = 0.0 if np.isnan(s_level) else s_level
    s_level = float(np.clip(s_level, 0.0, 1.0))

    nA = int(round(a * sr))
    nD = int(round(d * sr))
    nR = int(round(r * sr))

    fixed = nA + nD + nR
    if fixed > n and fixed > 0:
        scale = n / fixed
        nA = int(np.floor(nA * scale))
        nD = int(np.floor(nD * scale))
        nR = n - (nA + nD)


    nA = max(0, min(nA, n))
    nD = max(0, min(nD, n - nA))
    nR = max(0, min(nR, n - (nA + nD)))
    nS = n - (nA + nD + nR) 
    env_parts = []

    if nA > 0:
        env_parts.append(np.linspace(0.0, 1.0, nA, endpoint=False))
    if nD > 0:
        env_parts.append(np.linspace(1.0, s_level, nD, endpoint=False))

    if nS > 0:
        env_parts.append(np.full(nS, s_level))

    if nR > 0:
        if nS > 0:
            start = s_level
        elif nD > 0:
            start = s_level
        elif nA > 0:
            start = 1.0
        else:
            start = 1.0
        env_parts.append(np.linspace(start, 0.0, nR, endpoint=True))

    env = np.concatenate(env_parts) if env_parts else np.ones(n)
    return x * env[:n]


Heads up: This will likely be the most tedious question of this activity as it is a lot of array manipulation and edge case finding. 

Note: depending on your implementation, the following functions may come in handy:

`np.max` returns the maximum value in an array  
`np.min` returns the minimum value in an array  
`np.size` returns the total length of an array   
`np.where` returns index value(s) matching boolean conditions  
`np.zeros(n)` returns an array of zeros of length n  
`np.ones(n)` returns an array of ones of length n  
`np.full(n, val)` returns an array of values (val) of length n  

## 2. Create an amplitude modulation function

1. Write a function called amp_mod() for amplitude modulation. Your function should have the following arguments at minimum.

* carrier input signal (np.array)
* sampling rate (float)
* frequency of modulator (float)
* wave shape of the modulator (string: 'sine', 'square', 'tri', 'saw')
* boolean for polarity - should it be bipolar (True or False)

You need sampling rate to determine the proper relationship to length of the array and frequency.

Note: 
* modulator wave_shape --> should be sinusoid, square, sawtooth, or triangle (you may use `scipy.signal` library to make it easier - see: https://docs.scipy.org/doc/scipy/reference/signal.html and scroll to "Waveforms")  
* time --> the time or length of the carrier signal (modulator should exactly match)


2. Apply your function to the following scenarios.

a) Apply ring modulation with a square wave with a mod frequency of 50 Hz to a loaded audio file

In [ ]:
import numpy as np

def amp_mod(carrier: np.ndarray,
            sr: float = 44100.0,
            f_mod: float = 5.0,
            wave_shape: str = "sine", 
            bipolar: bool = False
            ) -> np.ndarray:
    x = np.asarray(carrier, dtype=float)
    if x.size == 0:
        return x

    sr = float(sr) if sr and sr > 0 else 44100.0
    f_mod = float(f_mod)


    if shape == "sine":
        m = np.sin(phase)

    elif shape == "square":
        m = np.sign(np.sin(phase))
        m[m == 0] = 1.0

    elif shape in ("tri", "triangle"):
        tf = f_mod * t
        m = 2.0 * np.abs(2.0 * (tf - np.floor(tf + 0.5))) - 1.0

    elif shape == "saw":
        tf = f_mod * t
        m = 2.0 * (tf - np.floor(tf + 0.5))

    else:
        raise ValueError("wave_shape must be one of: 'sine', 'square', 'tri', 'saw'")

    if not bipolar:
        m = 0.5 * (m + 1.0)

    if x.ndim == 1:
        return x * m
    elif x.ndim == 2:
        return x * m[:, None]
    else:
        raise ValueError("carrier must be 1D (n,) or 2D (n, channels)")



b) Apply amplitude modulation with a sine wave to a 200 Hz sine wave with an arbitrary modulator frequency. Plot the magnitude spectrum.

In [ ]:
sr = 44100
dur = 2.0
t = np.arange(int(sr * dur)) / sr

carrier_freq = 200.0
x = np.sin(2 * np.pi * carrier_freq * t)

f_mod = 30.0 
y_am = amp_mod(x, sr=sr, f_mod=f_mod, wave_shape="sine", bipolar=False) 

plot_mag_spectrum(y_am, sr, title=f"AM (sine), carrier=200 Hz, f_mod={f_mod} Hz")


c) Apply ring modulation with a sine wave to a 200 Hz sine wave with a modulator frequency such that the output is inharmonic. Plot the magnitude spectrum.

In [ ]:
f_mod_inharm = 73.0
y_ring_inharm = amp_mod(x, sr=sr, f_mod=f_mod_inharm, wave_shape="sine", bipolar=True)  

plot_mag_spectrum(y_ring_inharm, sr, title=f"Ring Mod (sine), carrier=200 Hz, f_mod={f_mod_inharm} Hz (inharmonic)")


d) Apply ring modulation with a sine wave to a 200 Hz sine wave with a modulator frequency such that the output only seems to have 1 frequency component. Plot the magnitude spectrum.

In [ ]:
f_mod_one = 200.0
y_ring_one = amp_mod(x, sr=sr, f_mod=f_mod_one, wave_shape="sine", bipolar=True)

plot_mag_spectrum(y_ring_one, sr, title=f"Ring Mod (sine), carrier=200 Hz, f_mod={f_mod_one} Hz (mostly 400 Hz + DC)")


## 3. Create an FM synthesis function 

1. Create a function called fmSynth() for FM synthesis (or more aptly PM synthesis). Your function should include the following arguments.

* Depth of modulation *or* modulation index (float) 
* Carrier frequency (float)
* Modulator frequency (float)
* time in seconds (float)

Your function should return a np.array with the synthesized sound

In [ ]:
import numpy as np

def fmSynth(I: float = 2.0,
            f_c: float = 220.0,
            f_m: float = 110.0,
            t: float = 2.0,
            sr: int = 44100) -> np.ndarray:
    sr = int(sr) if sr and sr > 0 else 44100
    t = float(t)
    n = int(round(sr * t))
    if n <= 0:
        return np.array([], dtype=float)

    time = np.arange(n) / sr
    # PM form: y(t) = sin(2π f_c t + I * sin(2π f_m t))
    y = np.sin(2 * np.pi * f_c * time + I * np.sin(2 * np.pi * f_m * time))
    return y


2) Using your function, try to create the following types of sounds. Try to understand the relation between the modulator and carrier in contributing to harmonic versus inharmonic sounds.

a) Inharmonic   

In [ ]:
sr = 44100
y_inharm = fmSynth(I=4.0, f_c=200.0, f_m=73.0, t=2.0, sr=sr)
y_inharm = normalize(y_inharm)

plot_mag_spectrum(y_inharm, sr, title="FM/PM Inharmonic: fc=200, fm=73, I=4", xlim=2000)


 b) harmonic where the carrier is the fundamental  


In [ ]:
y_harm_fc_fund = fmSynth(I=3.0, f_c=200.0, f_m=200.0, t=2.0, sr=sr)
y_harm_fc_fund = normalize(y_harm_fc_fund)

plot_mag_spectrum(y_harm_fc_fund, sr, title="FM/PM Harmonic (fc is fundamental): fc=200, fm=200, I=3", xlim=2000)


c) harmonic where the carrier is not the fundamental.  

In [ ]:
y_harm_not_fund = fmSynth(I=3.0, f_c=300.0, f_m=200.0, t=2.0, sr=sr)
y_harm_not_fund = normalize(y_harm_not_fund)

plot_mag_spectrum(y_harm_not_fund, sr, title="Not fundamental", xlim=2000)
